### Preprocessing (28 pts) ###

In [ ]:
import pandas as pd
import numpy as np
ga = pd.read_excel('GooglePlaystore.xlsx')
ga.shape

1. [3 pts] Often there are outliers which do not match the overall data type. There is one record in this data where the "Reviews" has value "3.0M" which does not match the rest of the data. Remove that record.

In [ ]:
ga = ga[ga.Reviews != '3.0M']
ga.shape

2. [4 pts] Remove rows where any of the columns has the value "Varies with device".

In [ ]:
for col in ga.columns:
    if ga[col].dtype != float:
        ga = ga[ga[col]!= 'Varies with device']
ga.shape

3. [5 pts] The values in the Android version column should be floats. Strip the trailing non-numeric characters from all values (ie. the words " and up"), so the result is a number. If there are multiple decimal places (eg. "x.y.z"), keep only the first two parts (eg "x.y"). For example, the value "4.1 and up" should be changed to "4.1". The value "4.5.6 and up" should be changed to "4.5". The value "5.6.7" should be changed to "5.6".

If there is a range (eg. 5.0 - 8.0), only consider the first number. For example, the value "5.0 - 8.0" should be changed to "5.0". The value "4.0.3 - 7.1.1" should be changed to "4.0".

In [ ]:
def change_ver(x):
    try:
        return x[:3]
    except:
        return x
ga['Android Ver'] = ga['Android Ver'].apply(change_ver)
ga.head()

4. [5 pts] The "Installs" column must have integer values. For values that have commas, remove the commas. For values that have a '+' at the end, remove the '+'. Keep only those rows that have an integer value after these edits.

In [ ]:
ga['Installs'].replace('\,','', regex=True, inplace=True) 
ga['Installs'].replace('\+','', regex=True, inplace=True) 
ga['Installs'] = ga['Installs'].astype(int)
ga.head()

5. [5 pts] For missing rating values, if the number of reviews is less than 100 and installations is less than 50000, remove the row. Else, fill the missing value with the average value (rounded to 2 decimal places) for the Category of that row.

In [ ]:
tmp = ga[((ga['Rating'].isna()) & (ga['Reviews'] < 100) & (ga['Installs'] < 50000))]
ga.drop(labels=tmp.index, inplace=True)
avg = ga.groupby('Category')['Rating'].mean()
dfs = []
for cate in ga['Category'].unique():
    tmp = ga[ga['Category']==cate]
    tmp['Rating'].fillna(round(avg[cate],2), inplace=True)
    dfs.append(tmp)
ga = pd.concat(dfs)
ga.head()

6. [6 pts] Preprocess the Size column to convert the "M" (millions) and "K" (thousands) values into integers. For instance, 8.7M should be converted to 8700000 and 2.4K should be converted to 2400.

In [ ]:
ga['Size'] = ga['Size'].replace({'M':'*1e6', 'K':'*1e3', 'k':'*1e3'}, regex=True).map(pd.eval).astype(int)
ga.head()

### Analysis (17 pts) ###

1. [4 pts] Describe (use DataFrame describe method) the category wise rating statistics. In other words, for each category, describe the statistics (count, mean, etc.) for ratings in that category.

In [ ]:
ga.groupby('Category')['Rating'].describe()

2. [11 pts] Extract all "Free" apps from the master data frame. Then write a function that, given a numeric column e.g 'Rating'), will create and return a dataframe for the top 3 free applications in each category based on that column. Call the function on each of these columns:
Rating (gives top 3 most highly rated applications in each category)
Installs (gives top 3 most installed applications in each category)
Reviews (gives top 3 most reviewed applications in each category)
You don't need to do anything explicit to break ties.

Each of the returned dataframes have Category and App for the first two columns, and one of Rating (for a.), Installs (for b.), and Reviews (for c.) as the third column.

In [ ]:
def free_data(type, ga):
    data_free = ga[ga['Type'] == 'Free']
    for cate in data_free.groupby('Category'):
        yield (cate[1].sort_values(by=type, ascending=False)[['Category', 'App', type]].iloc[:3])

for df in free_data('Rating', ga):
    print(df[['Category', 'App', 'Rating']])
    
for df in free_data('Installs', ga):
    print(df[['Category', 'App', 'Installs']])
    
for df in free_data('Reviews', ga):
    print(df[['Category', 'App', 'Reviews']])

3. [4 pts] Find the average, maximum and minimum price of the paid applications.

In [ ]:
print('Average:', ga[(ga['Type'] == 'Paid')]['Price'].mean())
print('Maximum:', ga[(ga['Type'] == 'Paid')]['Price'].max())
print('Minimum:', ga[(ga['Type'] == 'Paid')]['Price'].min())

### Visualization (16 pts) ###

In [ ]:
import matplotlib.pyplot as plt

1. [9 pts] In the genre column, break the string of genres into a list. For example, ‘Art & Design; Creativity’ should be [‘Art & Design’, Creativity’].
Count the number of applications per genre and display it using a pie chart.
Hint: Read about DataFrame.explode()

In [ ]:
ga['Genres'] = ga['Genres'].str.split(';')
ga = ga.explode('Genres')
genre = ga.groupby('Genres')['Genres'].count()
genre.plot.pie(figsize=(15,15), fontsize=7, autopct='%.1f%%')
plt.legend(loc=2, prop={'size': 8})
plt.show()

2. [7 pts] Display a box plot of ratings for "Business" and "Education" categories. The boxplots should be in the same plot.

In [ ]:
plt.boxplot([ga[ga['Category'] == 'BUSINESS']['Rating'],
             ga[ga['Category'] == 'EDUCATION']['Rating']], 
            labels=['BUSINESS','EDUCATION'])
plt.grid()
plt.show()